In [83]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from qiskit.synthesis import synth_clifford_depth_lnn
from qiskit.quantum_info import random_unitary
from qiskit.circuit.library import UnitaryGate
from qiskit import transpile
import numpy as np

In [84]:
# Challenge 1

qc = QuantumCircuit(2)

# Controlled-Y: control qubit 0, target qubit 1
qc.cy(0, 1)

# 3. Transpile to H, T, T†, CNOT
transpiled = transpile(
    qc,
    basis_gates=['h', 't', 'tdg', 'cx'],
    optimization_level=3
)

# 4. Count T gates
t_count = transpiled.count_ops().get('t', 0) + \
          transpiled.count_ops().get('tdg', 0)

print(f"T-count: {t_count}")



print(qc.draw())
print(transpiled.draw())

T-count: 8
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ Y ├
     └───┘
                                                  
q_0: ────────────────────────────────■────────────
     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┴─┐┌───┐┌───┐
q_1: ┤ T ├┤ T ├┤ T ├┤ T ├┤ T ├┤ T ├┤ X ├┤ T ├┤ T ├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘


In [85]:
qc2 = QuantumCircuit(2)
qc2.tdg(1)
qc2.tdg(1)
qc2.cx(0, 1)
qc2.t(1)
qc2.t(1)

print(qc2.draw())

                                  
q_0: ────────────────■────────────
     ┌─────┐┌─────┐┌─┴─┐┌───┐┌───┐
q_1: ┤ Tdg ├┤ Tdg ├┤ X ├┤ T ├┤ T ├
     └─────┘└─────┘└───┘└───┘└───┘


In [86]:
op1 = Operator(qc)
op2 = Operator(qc2)
print(op1.equiv(op2))
op1.equiv(transpiled)

True


True

In [ ]:
ts = QuantumCircuit(1)
ts.t(0)
ts.t(0)
ts.t(0)
ts.t(0)
ts.t(0)
ts.t(0)
ts_op = Operator(ts)

sdgs = QuantumCircuit(1)
sdgs.sdg(0)
sdgs_op = Operator(sdgs)

ts_op.equiv(sdgs_op)

True

In [87]:
stop

NameError: name 'stop' is not defined

In [ ]:
# Challenge 10

U = random_unitary(4, seed=42)  # unitary from challenge 10

# 2. Decompose to basic gates
# from qiskit.extensions import UnitaryGate
gate = UnitaryGate(U)
qc = QuantumCircuit(2)
qc.append(gate, [0,1])

# 3. Transpile to H, T, T†, CNOT
transpiled = transpile(
    qc,
    basis_gates=['h', 't', 'tdg', 'cx'],
    optimization_level=3
)

# 4. Count T gates
t_count = transpiled.count_ops().get('t', 0) + \
          transpiled.count_ops().get('tdg', 0)

print(f"T-count: {t_count}")


T-count: 251946
